<a href="https://colab.research.google.com/github/csw180/romanticpy/blob/main/W_pattern_fnder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 바닥신호를 잡기 위한  W 패턴 인식하는 로직

In [ ]:
import numpy as np
import pandas_datareader.data as web
import pandas as pd
import datetime
from collections import namedtuple

def  process_trickery(trickery_list) :
  index_list = []
  data_dict = {}    # way, angle 두개 요소
  data_dict['way'] = None
  data_dict['angle'] = None
  for row in trickery_list:
    tmp_way = data_dict['way']
    tmp_angle =data_dict['angle']
    if len(index_list) == 0 :
      index_list.append(row.Index)
      data_dict['way'] = row.way
      data_dict['angle'] = row.angle
      continue
    elif  ( (row.way == 'up') and (row.Low < float(data_dict['angle']) )) or \
          ( (row.way == 'down') and (row.High > float(data_dict['angle']) )) :
        index_list.clear()
        index_list.append(row.Index)
        data_dict['way'] = row.way
        data_dict['angle'] = row.angle   
  return pd.DataFrame(data_dict, index=index_list)


start = datetime.datetime(2021,10,1)
end = datetime.datetime(2022,2, 24)
# samsung_df = web.DataReader('005930', 'naver', start='2021-10-01', end='2022-02-24')
# samsung_df = web.DataReader('005930', 'naver',start='2021-01-01')
# samsung_df = web.DataReader('006890', 'naver')
df = web.DataReader('352820', 'naver')

df['ma5'] = df['Close'].rolling(window=5).mean()
df['ma20'] = df['Close'].rolling(window=20).mean()
df['vma20'] = df['Volume'].rolling(window=20).mean()
df['ma60'] = df['Close'].rolling(window=60).mean()
df['ma120'] = df['Close'].rolling(window=120).mean()
df['ma240'] = df['Close'].rolling(window=240).mean()
df['Open'] = df['Open'].astype(float, errors = 'raise')
df['High'] = df['High'].astype(float, errors = 'raise')
df['Low'] = df['Low'].astype(float, errors = 'raise')
df['Close'] = df['Close'].astype(float, errors = 'raise')
df['Volume'] = df['Volume'].astype(float, errors = 'raise')

conditionlist = [
    (df['Close'] > df['ma5']) & \
    (df['Close'].shift(1) < df['ma5'].shift(1)) & \
    (df['Close'].shift(2) < df['ma5'].shift(2)) & \
    (df['Close'].shift(3) < df['ma5'].shift(3))    ,
    (df['Close'] < df['ma5']) &\
    (df['Close'].shift(1) > df['ma5'].shift(1)) &\
    (df['Close'].shift(2) > df['ma5'].shift(2)) &\
    (df['Close'].shift(3) > df['ma5'].shift(3))]
choicelist1 = ['up', 'down']
choicelist2 = [df['Low'].rolling(4).min(),df['High'].rolling(4).max()]

df['way'] = np.select(conditionlist, choicelist1, default='')
df['angle'] = np.select(conditionlist, choicelist2, default='')
df = df.dropna()
df = df[df['way'] > '']
df = df[::-1]
# print(df.head(40))

df_copy = None
prev_way = None
trickery_list =[]
for row in df.itertuples():
  if (prev_way is None ) | (prev_way == row.way) :
    trickery_list.append(row)
    prev_way = row.way
  elif prev_way != row.way :
    if df_copy is None :
      df_copy = process_trickery(trickery_list).copy()
    else :
      df_copy = pd.concat([df_copy,process_trickery(trickery_list)])
    trickery_list.clear()
    trickery_list.append(row)
    prev_way = row.way
if  len(df) > 0 :
  if df_copy is None :
    df_copy = process_trickery(trickery_list).copy()
  else :
    df_copy = pd.concat([df_copy,process_trickery(trickery_list)])

print(type(df_copy))
print(df_copy.head(30))

